In [ ]:
# API token and CA crt

$serviceAccount = "cp-api-explorer"
# get right secret
$secretName = kubectl get serviceaccount cp-api-explorer -o json | ConvertFrom-Json `
    | % { $_.secrets } `
    | ? { $_.name.startsWith("$($serviceAccount)-token-") } `
    | select -First 1 | % { $_.name }
$secretName
# token from secretName
$token = kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data.token | base64 -d }
# ca.crt
$caCrtFilename = "./k8sca.crt"
kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data."ca.crt" | base64 -d } | Out-File $caCrtFilename
$caCrtFilename 
# API server URL
$apiServerUrl = kubectl config view -o json | jq -c '.clusters[] | select(.name == ""minikube"") | .cluster.server'
$apiServerUrl

In [ ]:
# test - name of first pod in default NS
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq -r '.items[0] | .metadata.name'

In [ ]:
# list of namespaces
$namespaces = /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq -r '.items[] | .metadata.name'
$namespaces

In [ ]:
# learn from masters - verbose mode of kubectl
kubectl get svc -v=8

In [ ]:
# verbose - list services (expect to fail - no permissions to access SERVICES!)
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/services" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `


In [ ]:
# all pods per namespace with IP address
$namespaces | % {
    $namespace = $_
    /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/$namespace/pods" `
        --header "Authorization: Bearer $token" --cacert $caCrtFilename `
        | ConvertFrom-Json | % { $_.items } | % {
            [PSCustomObject] @{
                namespace = $namespace;
                name = $_.metadata.name;
                ip = $_.status.podIPs[0].ip
            }
        }
}

In [ ]:
# all pods per namespace with IP address
$objects = $namespaces | % {
    $namespace = $_
    $pods = /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/$namespace/pods" `
        --header "Authorization: Bearer $token" --cacert $caCrtFilename `
        | ConvertFrom-Json | % { $_.items } | % {
            [PSCustomObject] @{
                namespace = $namespace;
                name = $_.metadata.name;
                ip = $_.status.podIPs[0].ip
            }
        }
    $ips = $pods | % { $_.ip }
    if ($ips.Length -gt 0) { 
        # Write-Host $namespace ": " $ips 
        [PSCustomObject] @{
            name = "ns-$namespace-ips";
            id = "id-ns-$namespace-ips";
            description = "IP adresses in namespace $namespace";
            ranges = $ips
        }
    }
}

$objects

In [ ]:
[PSCustomObject] @{
        "version" = "1.0";
        "description" = "Generic Data Center file example";
        "objects" = $objects
} | ConvertTo-Json -Depth 10 | jq .